# Export Bathymetry

Exports bathymetry tiles from a geometry of interest to GEE assets and GCS.
TODO: create logic in outside of notebook

In [1]:
from pathlib import Path
import sys
import time

import ee
import geemap
import geetools
import geojson

sys.path.append(str(Path.cwd().parent.parent))

from eobathymetry import bathymetry, tiler, ee_tiler, utils

## Create tiles based on input geometry

In [2]:
Map = geemap.Map(center=(52.97, 4.74), zoom=10)

aoi_json = """{
  "type": "Polygon",
  "coordinates": [
    [
      [
        4.726555204480136,
        52.79894952106581
      ],
      [
        5.382988309948886,
        53.2615577684405
      ],
      [
        5.226433134167636,
        53.48931215536743
      ],
      [
        4.770500516980136,
        53.41898585234949
      ],
      [
        4.270622587292636,
        52.91018589685636
      ],
      [
        4.726555204480136,
        52.79894952106581
      ]
    ]
  ]
}"""
bounds = ee.Geometry(geojson.loads(aoi_json))

## Plot tiles for quality control

In [ ]:
print(tiler.to_radians(10))
print(ee_tiler.to_radians(10).getInfo())

In [ ]:
print(tiler.zoom_to_scale(10))
print(ee_tiler.zoom_to_scale(10).getInfo())

In [ ]:
print(tiler.degrees_to_tiles(10, 10, 10)[1])
print(ee_tiler.degrees_to_tiles(10, 10, 10).get(1).getInfo())

In [ ]:
import math
print(math.pow(2, 10) - 30 - 1)
print(ee.Number(2).pow(ee.Number(10)).subtract(30).subtract(1).getInfo())

In [ ]:
print(tiler.get_tile_bounds(10, 30, 10))
print(ee_tiler.get_tile_bounds(10, 30, 10).getInfo())

In [ ]:
print(tiler.pixels_to_meters(1298379, 129387198, 11))
print(ee_tiler.pixels_to_meters(1298379, 129387198, 11).getInfo())

In [ ]:
print(tiler.get_tiles_for_geometry(bounds, 10).size().getInfo())
print("------------------")
print(ee_tiler.get_tiles_for_geometry(bounds, 10).size().getInfo())

In [3]:
def add_tile_bounds(zoom):
    tiles = ee_tiler.get_tiles_for_geometry(bounds, zoom)
    Map.addLayer(tiles.style(width=max(1, 10 - zoom), fillColor= "00000022"), {}, "tiles " + str(zoom))

In [4]:
list(map(add_tile_bounds, [8, 9, 10, 11, 12]))
Map

Map(center=[52.97, 4.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

## Export to earth engine assets
Create tiles to export. Then export based tiles and given zoom levels.

In [19]:
from typing import Any, Dict, List
from dateutil.parser import parse
import functools

In [123]:
def get_tile_bathymetry(tile: ee.Feature, start: ee.String, stop: ee.String) -> ee.Image:
    bounds: ee.Geometry = ee.Feature(tile).geometry().bounds(1)
    scale: ee.Number = ee_tiler.zoom_to_scale(ee.Number.parse(tile.get("zoom")))
    sdb: Bathymetry = bathymetry.Bathymetry()
    zoom: ee.String = ee.String(tile.get("zoom"))
    tx: ee.String = ee.String(tile.get("tx"))
    ty: ee.String = ee.String(tile.get("ty"))
    tile_name: ee.String = ee.String("z").cat(zoom).cat("_x").cat(tx).cat("_y").cat(ty).replace("\.\d+", "", "g")
    img_fullname: ee.String = ee.String(tile_name).cat("_t").cat(ee.Date(start).millis().format())
    tile.set(tile_name)
        
    image: ee.Image = sdb.compute_inverse_depth(
                bounds=bounds,
                start=start,
                stop=stop,
                filter_masked=True,
                scale=ee_tiler.zoom_to_scale(ee.Number.parse(tile.get("zoom"))).multiply(5),
    )
    image = image.set(
        "fullname", img_fullname,
        "system:time_start", ee.Date(start).millis(),
        "system:time_stop", ee.Date(stop).millis(),
        "zoom", zoom,
        "tx", tx,
        "ty", ty
    )
    return image
#     ee.batch.Export.image.toAsset(
#         image,
#         description="export SDB tiles",
#         assetId=ee.String(asset_path).cat(tile_fullname),
# #         pyramidingPolicy=None,
#         dimensions=None,
#         region=bounds,
#         scale=None,
#         crs=None,
#         crsTransform=None,
#         maxPixels=None
#     )

In [128]:
import pprint
def export_sdb_tiles(
    num_tiles: int,
    scale: int,
    tile_list: ee.List,
    sdb_tiles: ee.ImageCollection,
    asset_path: str,
    task_list: List[ee.batch.Task],
    overwrite: bool
) -> List[ee.batch.Task]:
    for i in range(num_tiles):
        # get tile
        temp_tile: ee.Feature = ee.Feature(tile_list.get(i))
        
        # filter imagecollection based on tile
        filtered_ic: ee.ImageCollection = sdb_tiles \
            .filterMetadata("tx", "equals", ee.String(temp_tile.get("tx"))) \
            .filterMetadata("ty", "equals", ee.String(temp_tile.get("ty"))) \
            .filterMetadata("zoom", "equals", ee.String(temp_tile.get("zoom")))
        # if filtered correctly, only a single image remains
        img: ee.Image = ee.Image(filtered_ic.first())  # have to cast here
        # Export image
        img_name: str = img.get("fullname").getInfo()
        asset_id: str = f"{asset_path}/{img_name}"
        asset: Dict[str, Any] = ee.data.getInfo(asset_id)
        if asset and overwrite:
            ee.data.deleteAsset(asset_id)
        elif asset:
            raise RuntimeError(f"asset {asset} already exists")
        task: ee.batch.Task = ee.batch.Export.image.toAsset(
            img,
            assetId=asset_id,
            description=img_name,
            region=temp_tile.geometry(),
            scale=scale
        )
        task.start()
        print(f"exporting {img_name} to {asset_id}")
        task_list.append(task)
    return task_list

In [129]:
def export_tiles_to_assets(
    asset_path: str,
    geometry: ee.Geometry,
    sdb: bathymetry.Bathymetry,
    zoom: int,
    start: str,
    stop: str,
    step_months: int = 3,
    window_years: int = 2,
    overwrite: bool = False
):
    
    def create_year_window(year: ee.Number, month: ee.Number) -> ee.Dictionary:
        t: ee.Date = ee.Date.fromYMD(year, month, 1)
        d_format: str = "YYYY-MM-dd"
        return ee.Dictionary({"start": t.format(d_format), "stop": t.advance(window_years, 'year').format(d_format)})
        
    dates: ee.List = ee.List.sequence(parse(start).year, parse(stop).year).map(
        lambda year: ee.List.sequence(1, 12, step_months).map(functools.partial(create_year_window, year))
    ).flatten()
    
    # Get tiles
    tiles: ee.FeatureCollection = ee_tiler.get_tiles_for_geometry(geometry, ee.Number(zoom))

    scale: float = ee_tiler.zoom_to_scale(zoom).getInfo()
    task_list: List[ee.batch.Task] = []
    num_tiles: int = tiles.size().getInfo()
    tile_list: ee.List = tiles.toList(num_tiles)
    
#     buffered_bounds: ee.Geometry = tiles.first().geometry().buffer(ee_tiler.zoom_to_scale(zoom)).bounds()
#     Map.addLayer(get_tile_bathymetry(tiles.first(), start, stop).clip(buffered_bounds), {}, "preview SDB")
    for date in dates.getInfo():
        sdb_tiles: ee.ImageCollection = tiles.map(lambda tile: get_tile_bathymetry(tile, ee.String(date["start"]), ee.String(date["stop"])))
#         image = sdb_tiles.first()
#         ee.batch.Export.image.toAsset(
#             image,
#             assetId=f"{asset_path}/{image.get('name').getInfo()}",
#             description=image.get('name').getInfo(),
#             region=tile.first().geometry(),
#             scale=scale
#         )
#         return

        # Now export tiles, based off https://github.com/gee-community/gee_tools/blob/master/geetools/batch/imagecollection.py#L166
        export_sdb_tiles(num_tiles, scale, tile_list, sdb_tiles, asset_path, task_list, overwrite)
            
#     print(bathymetry_tiles.first().getInfo())
#     batch.imagecollection.toAsset(
#         bathymetry_tiles,
#         asset_path,
#         namePattern=None,
#         scale=ee_tiler.zoom_to_scale(ee.Number(zoom)).getInfo(),
#         region=None,
#         create=True,
#         verbose=False,
#         datePattern='yyyyMMdd',
#         extra=None
#     )

In [130]:
user_name = ee.data.getAssetRoots()[0]["id"].split("/")[-1]
asset_path = f"users/{user_name}/eo-bathymetry"
# asset_dict = ee.data.listAssets({"parent": f"projects/earthengine-legacy/assets/users/{user_name}"})
# if any([f"users/{user_name}/eo-bathymetry" in asset["id"] for asset in asset_dict["assets"]]):
#     ee.data.deleteAsset(asset_path)
ee.data.create_assets(asset_ids=[asset_path], asset_type="Folder", mk_parents=True)

export_tiles_to_assets(asset_path, bounds, bathymetry.Bathymetry(), 8, "2020-01-01", "2021-01-01", overwrite=True)

Asset users/jaapel/eo-bathymetry already exists.
exporting z8_x131_y82_t1577836800000 to users/jaapel/eo-bathymetry/z8_x131_y82_t1577836800000
exporting z8_x131_y83_t1577836800000 to users/jaapel/eo-bathymetry/z8_x131_y83_t1577836800000
exporting z8_x131_y82_t1585699200000 to users/jaapel/eo-bathymetry/z8_x131_y82_t1585699200000
exporting z8_x131_y83_t1585699200000 to users/jaapel/eo-bathymetry/z8_x131_y83_t1585699200000
exporting z8_x131_y82_t1593561600000 to users/jaapel/eo-bathymetry/z8_x131_y82_t1593561600000
exporting z8_x131_y83_t1593561600000 to users/jaapel/eo-bathymetry/z8_x131_y83_t1593561600000
exporting z8_x131_y82_t1601510400000 to users/jaapel/eo-bathymetry/z8_x131_y82_t1601510400000
exporting z8_x131_y83_t1601510400000 to users/jaapel/eo-bathymetry/z8_x131_y83_t1601510400000
exporting z8_x131_y82_t1609459200000 to users/jaapel/eo-bathymetry/z8_x131_y82_t1609459200000
exporting z8_x131_y83_t1609459200000 to users/jaapel/eo-bathymetry/z8_x131_y83_t1609459200000
exporting z